In [2]:
import pyspark

In [15]:
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/08/07 16:29:17 WARN Utils: Your hostname, DESKTOP-INRF453 resolves to a loopback address: 127.0.1.1; using 172.26.66.126 instead (on interface eth0)
22/08/07 16:29:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/07 16:29:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sdf = spark.read.parquet('../../ADS_Project_1/raw_data/tlc_data')

In [7]:
sdf.limit(10)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-03-01 11:13:08,2022-03-01 11:24:35,1.0,2.4,1.0,N,90,209,2,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,0.0
1,2022-03-01 11:47:52,2022-03-01 12:00:08,1.0,2.2,1.0,N,148,234,2,10.5,3.0,0.5,0.0,0.0,0.3,14.3,2.5,0.0
2,2022-03-01 11:02:46,2022-03-01 11:46:43,1.0,19.78,2.0,N,132,249,1,52.0,0.0,0.5,11.06,0.0,0.3,67.61,2.5,1.25
2,2022-03-01 11:52:43,2022-03-01 12:03:40,2.0,2.94,1.0,N,211,66,1,11.0,0.5,0.5,4.44,0.0,0.3,19.24,2.5,0.0
2,2022-03-01 11:15:35,2022-03-01 11:34:13,1.0,8.57,1.0,N,138,197,1,25.0,0.5,0.5,5.51,0.0,0.3,33.06,0.0,1.25
1,2022-03-01 11:11:57,2022-03-01 11:53:05,2.0,14.0,1.0,N,132,33,1,43.5,1.75,0.5,9.2,0.0,0.3,55.25,0.0,1.25
2,2022-03-01 11:05:11,2022-03-01 11:08:22,1.0,0.61,1.0,N,166,151,1,4.5,0.5,0.5,1.0,0.0,0.3,6.8,0.0,0.0
2,2022-03-01 11:30:56,2022-03-01 11:46:21,1.0,2.83,1.0,N,74,238,1,13.0,0.5,0.5,3.7,0.0,0.3,18.0,0.0,0.0
2,2022-03-01 11:30:28,2022-03-01 11:30:36,1.0,0.1,1.0,N,145,145,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,0.0,0.0
2,2022-03-01 11:30:28,2022-03-01 11:30:36,1.0,0.1,1.0,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,0.0


In [8]:
sdf.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



# Preprocessing Steps -


## 1. Typecasting
Making sure that all columns are appropriately typed

## 2. Getting rid of unrelated data (as per Data Dictionary)
We won't care about some fields for the purposes of our analysis, so need to drop them

## 3. Handling of Missing Values
If NaN / null values are present, need to deal with them appropriately - imputation, deletion, etc

## 4. Outlier Removal / Analysis
Outliers have to be dealt with appropriately, as they can skew data


### Typecasting

There are several columns that we need to adjust the type for. They'll be listed below with their justifications
- VendorID: Inspection of the data revealed that this only takes integer values, so it should be changed to an int
- passenger_count: Should be changed to int since # of people is an integer
- RatecodeID: Similar to VendorID

In [13]:
# I want to check if any taxi drivers have reported half a person -
columns = ['passenger_count']
for column in columns:
    sdf.select(column).distinct().show()
    
# Can confirm that only integers are reported

+---------------+
|passenger_count|
+---------------+
|            8.0|
|            0.0|
|            7.0|
|           null|
|            1.0|
|            4.0|
|            3.0|
|            2.0|
|            6.0|
|            5.0|
|            9.0|
+---------------+



# How come this code doesn't work?

sdf.select(column).distinct().show()

In [16]:
columns = ['VendorID', 'passenger_count', 'RatecodeID']
for column in columns:
    sdf = sdf.withColumn(
        column,
        F.col(column).astype('INT')
    )

In [17]:
sdf.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### Converting PU/DOLocation to Geometry 
Code attributed to tute2

In [21]:
sdf.select(F.min('fare_amount'))

min(fare_amount)
-2564.0


In [ ]:
aggregated_results = sdf \
                    .groupBy("passenger_count") \
                    .agg(
                        F.mean("total_amount").alias("avg_trip_amount_usd"),
                        F.max("trip_distance").alias("max_trip_distance_miles")
                    ) \
                    .orderBy("passenger_count")

aggregated_results.show()